In [1]:
import random
import torch
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score, mean_absolute_error
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [2]:
def fix_random(seed: int) -> None:
    """Fix all the possible sources of randomness.

    Args:
        seed: the seed to use.
    """
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True  # slower

In [3]:
SEED = 1038893

fix_random(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))

root = "../../data/ml-25m"

Device: cpu


## Data Acquisition

In [4]:
def get_data_from_csv(file: str, nrows=None):
    if nrows:
        df = pd.read_csv(f"{root}/{file}", nrows=nrows)
    else:
        df = pd.read_csv(f"{root}/{file}")
    print(f"Loaded ml-25m data: {root}/{file}")

    return df

def get_ratings_from_csv(nrows=None):
    path = "ratings.csv"

    data = get_data_from_csv(path, nrows)
    # data = get_data_from_csv(path)

    data.drop("timestamp", axis=1, inplace=True)
    # todo: drop user id
    #data.drop("userId", axis=1, inplace=True)

    return data

def get_tag_relevances_from_csv(nrows=None):
    path = "genome-scores.csv"

    return get_data_from_csv(path, nrows)

In [5]:
genome_scores_path = "genome-scores.csv"
ratings_path = "ratings.csv"

In [6]:
ratings = get_ratings_from_csv()
print(ratings)

Loaded ml-25m data: ../../data/ml-25m/ratings.csv
          userId  movieId  rating
0              1      296     5.0
1              1      306     3.5
2              1      307     5.0
3              1      665     5.0
4              1      899     3.5
...          ...      ...     ...
25000090  162541    50872     4.5
25000091  162541    55768     2.5
25000092  162541    56176     2.0
25000093  162541    58559     4.0
25000094  162541    63876     5.0

[25000095 rows x 3 columns]


In [7]:
genome_scores = get_tag_relevances_from_csv()
print(genome_scores)

Loaded ml-25m data: ../../data/ml-25m/genome-scores.csv
          movieId  tagId  relevance
0               1      1    0.02875
1               1      2    0.02375
2               1      3    0.06250
3               1      4    0.07575
4               1      5    0.14075
...           ...    ...        ...
15584443   206499   1124    0.11000
15584444   206499   1125    0.04850
15584445   206499   1126    0.01325
15584446   206499   1127    0.14025
15584447   206499   1128    0.03350

[15584448 rows x 3 columns]


In [8]:
# films = get_data_from_csv(f"{root}/{ratings}")[]


# DATA VISUALIZATION


In [9]:
            # # Fill in missing values with zeros
            # X.fillna(0, inplace=True)


# FARE TEST CON AVG, STD_DEV

In [13]:
def addColumnOperation(ratings,X):
     # Compute the mean rating for each user
     count_rating = ratings.groupby('movieId', as_index=False)['rating'].count()
     std= ratings.groupby('movieId', as_index=False)['rating'].std()
     std.fillna(0, inplace=True)
     min_ratings= ratings.groupby('movieId', as_index=False)['rating'].min()
     max_ratings= ratings.groupby('movieId', as_index=False)['rating'].max()
     median= ratings.groupby('movieId', as_index=False)['rating'].median()
     operation = pd.DataFrame({'movieId':count_rating['movieId'],'count_rating': count_rating['rating'], 'std': std['rating'], 'min': min_ratings['rating'], 'max': max_ratings['rating'], 'median': median['rating']}) 
     X = pd.merge(X, operation, on='movieId')
     X.drop("movieId", axis=1, inplace=True)
     return X
    

In [14]:
class LinearRegressionModel:
    def __init__(self, ratings, relevance, seed=42):
        #! df['rating'] = df['rating'].astype('float16')

        # Reduce genome-score size

        # Pivot the relevance DataFrame to create a matrix of tag relevance scores for each movie
        relevance_matrix = relevance.pivot_table(index='movieId', columns='tagId', values='relevance', fill_value=0)

        
        # Merge the ratings and relevance data
        mean_ratings = ratings.groupby('movieId', as_index=False)['rating'].mean()
        X = mean_ratings.merge(relevance_matrix, on='movieId')
        X = addColumnOperation(ratings,X)
        X.columns = X.columns.astype(str)
        ratings = None  
        # mescolare le righe del DataFrame
        #X = X.sample(frac=1).reset_index(drop=True)
        
        y = X['rating']
        X = X.drop('rating', axis=1)
        

        print(X.columns)
        # Split the data into training and testing sets

        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
        print(self.X_train.shape, self.X_test.shape, self.y_train.shape, self.y_test.shape)
        # Initialize the Gaussian Naive Bayes model
        self.model = LinearRegression()

        # Set the hyperparameters for tuning
        self.params = {
            'fit_intercept': [True, False],
            'copy_X': [True, False],
            'positive': [True, False]
        }

    def train(self, num_epochs=10):
        # Fit the model to the training data
        for epoch in range(num_epochs):
            self.model.fit(self.X_train, self.y_train)
            self.test()
            print(f"Epoch {epoch + 1}/{num_epochs} completed")

     # Hyperparameter tuning
    def tuning(self):
        grid_search = GridSearchCV(self.model, param_grid=self.params, cv=cv, scoring=scoring)
        grid_search.fit(self.X_train, self.y_train)
        self.model = grid_search.best_estimator_
        print(f"Best parameters: {grid_search.best_params_}")
        print(f"Best score: {grid_search.best_score_:.4f}")
        
        # Re-initialize the model with the best hyperparameters
        #self.model = LinearRegression(**grid_search.best_params_)


    def test(self):
        # Predict ratings for the test data
        y_pred = self.model.predict(self.X_test)

        # Compute the mean squared error
        
        mse = mean_squared_error(self.y_test, y_pred)
        rmse = mean_squared_error(self.y_test, y_pred, squared=False)
        r2 = r2_score(self.y_test, y_pred)
        mae = mean_absolute_error(self.y_test, y_pred)

        print(f"MSE: {mse} RMSE: {rmse} R2: {r2} MAE: {mae}")
        score = self.model.score(self.X_test, self.y_test)
        print(f"Score:{score}")
        print("Intercept: "+str( self.model.intercept_))
        print("Coefficients: "+ str(self.model.coef_))


In [15]:
lr = LinearRegressionModel(ratings, genome_scores, SEED)

RangeIndex(start=0, stop=1133, step=1)
(11052, 1133) (2764, 1133) (11052,) (2764,)


In [16]:
#lr.hyperparameter_tuning()

In [17]:
lr.train(4)

MSE: 0.004342858099256041 RMSE: 0.06590036494023414 R2: 0.9811336807031474 MAE: 0.051444139118032264
Score:0.9811336807031474
Intercept: 3.011937458364656
Coefficients: [ 0.00494105 -0.05823666  0.00305027 ...  0.014653    0.0728805
  0.54850339]
Epoch 1/2 completed
MSE: 0.004342858099256041 RMSE: 0.06590036494023414 R2: 0.9811336807031474 MAE: 0.051444139118032264
Score:0.9811336807031474
Intercept: 3.011937458364656
Coefficients: [ 0.00494105 -0.05823666  0.00305027 ...  0.014653    0.0728805
  0.54850339]
Epoch 2/2 completed


In [18]:
lr.test()

MSE: 0.004342858099256041 RMSE: 0.06590036494023414 R2: 0.9811336807031474 MAE: 0.051444139118032264
Score:0.9811336807031474
Intercept: 3.011937458364656
Coefficients: [ 0.00494105 -0.05823666  0.00305027 ...  0.014653    0.0728805
  0.54850339]
